In [1]:
import sys, os, re, copy
import dill as pickle # this serializes all the functions inside the quantification dict
import numpy as np
import scipy as sp
from scipy.optimize import newton, minimize, fsolve
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import matplotlib.colors as mcolors
palette = list(mcolors.TABLEAU_COLORS.keys())
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid", {
 'axes.spines.bottom': True,
 'axes.spines.left': True,
 'axes.spines.right': True,
 'axes.spines.top': True
})
sns.set(font_scale=1)
palette = list(mcolors.TABLEAU_COLORS.keys())
sns.set_theme(style="ticks", palette="muted")
sns.set_context("talk")

## Quantification of free amino acids in 143B and H1299 cells
Six well plates of 143B and H1299 cells were cultured in DMEM, 10% dialyzed FBS, no pyruvate with either vehicle or 1 mM Asn.
At close to confluency, the top row of cells were washed twice with cold saline and metabolites extracted using 1 mL 80% MeOH.
The bottom row was counted using a Coulter counter to determine the total cell volume.
The metabolite extract was spun down and 500 uL was dried.
Samples were then reconstituted in 40 uL U-15N, U-13C amino acid internal standards (CAA) in 80 % MeOH and quantified using LCMS.
The dilution factor was calculated based on the mean total cell volume determined by Coulter counter.

Here these samples are processed to calculate the intracellular concentration of each amino acid.

In [2]:
def impute_conc(piece_wise_fit_metab, response_ratio):
    '''
    This function imputes the concentration from a response ratio.
    '''
    response_ratio_range = np.array(list(piece_wise_fit_metab.keys()))
    mask_range = [response_ratio >= min_v and response_ratio <= max_v for max_v, min_v in response_ratio_range]
    k = tuple(response_ratio_range[mask_range][0])
    conc = piece_wise_fit_metab[k](response_ratio)
    estimator = 'intrapolation'
    if 0 in k:
        estimator = 'extrapolation under'
    elif np.inf in k:
        estimator = 'extrapolation over'
    return(conc, estimator)

In [3]:
### Read quantification function ###
dict_pickle_fnam = 'input/AA-quant_oct_20.pickle'
with open(dict_pickle_fnam, 'rb') as handle:
    piece_wise_fit_quant = pickle.load(handle)

### Read measurements ###
### Replace all N/F with 0 before start ###
esheet_dict_mes = pd.read_excel('input/H1299-143B_intra-Asp.xlsx', sheet_name=None)
annotation_df = pd.read_excel('input/annotations.xlsx')
metab_dict_mes = dict()
metab_names_mes = list()
for k in esheet_dict_mes.keys():
    if 'U-13C' not in k:
        metab_names_mes.append(k)
        metab_dict_mes[k] = copy.deepcopy(esheet_dict_mes[k])
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Area'] / metab_dict_mes[k]['ISTD Response']
        metab_dict_mes[k]['Response Ratio'] = metab_dict_mes[k]['Response Ratio'].fillna(0).replace(np.inf, 0)
        metab_dict_mes[k]['Sample_name'] = [fn.split('_')[-1] for fn in metab_dict_mes[k]['Filename']]
        # Add annotations:
        metab_dict_mes[k] = metab_dict_mes[k].merge(annotation_df, left_on='Sample_name', right_on='Sample_name')
        metab_dict_mes[k] = metab_dict_mes[k].drop(['Flag Details', 'Theoretical Amt', 'Filename', 'Type', 'RT', 'Sample ID', 'Metformin', 'ISTD Response'], axis=1)

In [4]:
### Impute concentration and add to metabolite dataframe ###
rr_mes = dict() # for plotting 
imp_conc_mes = dict() # for plotting
for metab in metab_names_mes[:]:
    # Assign imputed concentration:
    if metab in piece_wise_fit_quant:
        conc_list = list()
        estimator_list = list()
        for rr in metab_dict_mes[metab]['Response Ratio'].values:
            conc, estimator = impute_conc(piece_wise_fit_quant[metab], rr)
            conc_list.append(conc)
            estimator_list.append(estimator)
        metab_dict_mes[metab]['imputed_sample_conc'] = conc_list
        metab_dict_mes[metab]['imputed_sample_estimator'] = estimator_list

        # Extract data for plotting:
        if metab in rr_mes:
            rr_mes[metab].extend(list(metab_dict_mes[metab]['Response Ratio'].values))
            imp_conc_mes[metab].extend(list(metab_dict_mes[metab]['imputed_sample_conc'].values))
        else:
            rr_mes[metab] = list(metab_dict_mes[metab]['Response Ratio'].values)
            imp_conc_mes[metab] = list(metab_dict_mes[metab]['imputed_sample_conc'].values)

In [5]:
# Move the measured concentration to new dataframe:
df = copy.deepcopy(metab_dict_mes[metab_names_mes[0]]).drop(['Compound', 'Formula', 'Response Ratio', 'Area', 'Formula', 'Proliferation_rate', 'imputed_sample_conc', 'imputed_sample_estimator'], axis=1)
for nam in piece_wise_fit_quant.keys():
    if nam in metab_dict_mes:
        assert(metab_dict_mes[nam]['Sample_name'].equals(df['Sample_name']))
        df[nam] = metab_dict_mes[nam]['imputed_sample_conc']
# Find the mean over 3 replicates:
df_mean = df.groupby(['Cell_line', 'Salvage_mix']).mean().reset_index()

# Calculate the intracellular concentration:
df_mean_cell_conc = copy.deepcopy(df_mean)
AA_cols = df_mean_cell_conc.columns.values[3:]
df_mean_cell_conc.loc[:, AA_cols] = df_mean_cell_conc.loc[:, AA_cols].div(df_mean_cell_conc['Dilution_factor'], axis=0)
df_mean_cell_conc = df_mean_cell_conc.drop(['Dilution_factor'], axis=1)
# Write results to an Excel file:
df_mean_cell_conc.to_excel('output/AA_intra-conc.xlsx')